In [76]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import codecs
import json
import html

In [77]:
domain = 'https://www.indeed.com'
jobLinkToken = 'class="jobtitle">'
nextPageToken = '<link rel="next" href="'
companyNameToken = '<div class="icl-u-lg-mr--sm icl-u-xs-mr--xs">'
descriptionToken = '<div class="jobsearch-JobComponent-description icl-u-xs-mt--md">'
descriptionEnd = '</div><div class="jobsearch-JobDescriptionTab-content">'
locationToken = '<div class="icl-u-lg-mr--sm icl-u-xs-mr--xs">-</div><div>'
urlToken = '<span id="originalJobLinkContainer" class="icl-u-lg-inline icl-us-xs-hide"> - <a target=_blank rel=noopener&#32;noreferrer href='
urlEndToken = '>original job</a>'
jobsCrawled = 0

In [78]:
def getHtmlText (link):
    return str(urlopen(link).read())

def getJobLinks (indexPage):
    links = []
    titleBlocks = indexPage.split(jobLinkToken)[1:]
    for titleBlock in titleBlocks:
        try:
            anchor = titleBlock.split('</h2>')[0]
            rawPath = anchor.split('href="')[1]
            path = rawPath.split('"')[0]
            links.append(domain + path)
        except:
            continue
    return links

def extractJobInfo (jobPage):
    title = jobPage.split('<title>')[1].split(' - ')[0]
    company = jobPage.split(companyNameToken)[1].split('</div>')[0]
    location = jobPage.split(locationToken)[1].split('</div>')[0]
    rawDescripton = jobPage.split(descriptionToken)[1].split(descriptionEnd)[0]
    rawDescription = BeautifulSoup(rawDescripton).text
    description = (eval(repr(rawDescription).replace('\\\\', '\\')).encode('ascii', 'ignore')).decode("utf-8")
    rawLink = jobPage.split(urlToken)
    if len(rawLink) > 1:
        link = html.unescape(rawLink[1].split(urlEndToken)[0])
    else:
        link = 'Not Available'
    return {'title' : title, 'company' : company, 'location' : location, 'job url' : link, 'description' : description}

def getNextPageLink(indexPage):
    path = indexPage.split(nextPageToken)[1].split('" />')[0]
    return domain + path

In [79]:
def crawl (startingLink, pagesNeeded):
    pagesCrawled = 0
    pagesSkipped = 0
    indexLink = startingLink
    jobsInfo = []
    companiesInfo = []

    while pagesCrawled < pagesNeeded:  
        indexPage = getHtmlText(indexLink)
        jobLinks = getJobLinks(indexPage)
        print('\nCrawling index page: ' + indexLink + '\n')

        for jobLink in jobLinks:
            print('Crawling job link: ' + jobLink)
            try:
                jobPage = getHtmlText(jobLink)
            except:
                print('\nAn invalid job link appears.')
                pagesSkipped += 1
                print(str(pagesSkipped) + ' jobs skipped.\n')
            try:
                jobInfo = extractJobInfo(jobPage)
                jobsInfo.append(jobInfo)
            except:
                print('\nA non-crawlable pages occurs.')
                pagesSkipped += 1
                print(str(pagesSkipped) + ' jobs skipped.\n')

            #companyInfo = extractCompanyInfo(jobLink)
            #companiesInfo.append(companyInfo)
        indexLink = getNextPageLink(indexPage)
        print('\n' + str(pagesCrawled + 1) + ' pages have been crawled.\n')
        pagesCrawled += 1      
    return jobsInfo

def writeJson (name, data):
    with open(name, 'w') as fout:
        json.dump(data, fout)
    jobsCrawled += len(data)
    print(jobsCrawled)

In [80]:
software_data = crawl('https://www.indeed.com/jobs?q=software&l=Anywhere', 15)
writeJson('software jobs.json', software_data)


Crawling index page: https://www.indeed.com/jobs?q=software&l=Anywhere

Crawling job link: https://www.indeed.com/rc/clk?jk=2d08ad9edef643da&fccid=134b1f874a60ea79&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=9bacd95cebf2705e&fccid=c3dda9f226ad7226&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=ac91d880a1905565&fccid=fe2d21eef233e94a&vjs=3
Crawling job link: https://www.indeed.com/company/Spotlight--Education/jobs/Software-Engineer-e55f6b33f30dda3d?fccid=1c8532c4c4d3199c&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=4fdc702a46bbdd1a&fccid=a5b4499d9e91a5c6&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=c50fafce089cd1f6&fccid=86e9be6ce380173e&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=d62d554598f82dec&fccid=a5b4499d9e91a5c6&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=8167ae6417a6009e&fccid=de71a49b535e21cb&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=1bf9aaf7196d1e30&fccid=a5b4499d9e91a5c6&vjs=3
Cra

Crawling job link: https://www.indeed.com/company/QABASA/jobs/Entry-Level-QA-Analyst-156ea1246e157ac2?fccid=281a90d139276cca&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=10c17db7f3bee430&fccid=1639254ea84748b5&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=2141c0830e009a8e&fccid=1b41cc7ae60b8689&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=5c9b8fb7dd80b91b&fccid=fb74c70e196043bc&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=8ae7ed7a66090df7&fccid=6824fc1f087bd63e&vjs=3
Crawling job link: https://www.indeed.com/company/6D-Global/jobs/QA-Software-Engineer-5a012f843c7e1bc7?fccid=a386b2f670d93a46&vjs=3
Crawling job link: https://www.indeed.com/company/EGear-Media/jobs/Entry-Level-Junior-Software-Developer-a9d7fa47ae2c63f5?fccid=f19121e3465ff2c6&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=b497a45308de9a8a&fccid=734cb5a01ee60f80&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=dd23ad7e8bfdc353&fccid=f057e04c37cca134&vjs=


14 pages have been crawled.


Crawling index page: https://www.indeed.com/jobs?q=software&l=Anywhere&start=140

Crawling job link: https://www.indeed.com/company/Ginolis/jobs/Software-Application-Engineer-b8691cb041171027?fccid=a511ea0837d82db7&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=7a8ae35a0247fc8b&fccid=59c8e3e7017ea02a&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=28e480473dacfe25&fccid=535f108e774016a4&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=fc3148f4f95041af&fccid=a3163e1f583839d0&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=faf4f9636ac25b0f&fccid=11619ce0d3c2c733&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=5de82f1c821f50c6&fccid=9744e569304da4d3&vjs=3
Crawling job link: https://www.indeed.com/rc/clk?jk=476ea5bc57fceb02&fccid=efafbed82ace95b2&vjs=3

A non-crawlable pages occurs.
5 jobs skipped.

Crawling job link: https://www.indeed.com/rc/clk?jk=a1c586df60ec584f&fccid=8d06889afb939460&vjs=3
Crawling job l

UnboundLocalError: local variable 'jobsCrawled' referenced before assignment

In [ ]:
hardware_data = crawl('https://www.indeed.com/jobs?q=hardware&l=Anywhere', 15)
writeJson('hardware jobs.json', hardware_data)

In [ ]:
HR_data = crawl('https://www.indeed.com/jobs?q=human+resources&l=Anywhere', 15)
writeJson('HR jobs.json', HR_data)

In [ ]:
government_data = crawl('https://www.indeed.com/jobs?q=government&l=Anywhere', 15)
writeJson('government jobs.json', government_data)

In [ ]:
accounting_data = crawl('https://www.indeed.com/jobs?q=accounting&l=Anywhere', 15)
writeJson('accounting jobs.json', accounting_data)

In [ ]:
healthcare_data = crawl('https://www.indeed.com/jobs?q=healthcare&l=Anywhere', 15)
writeJson('healthcare jobs.json', healthcare_data)

In [ ]:
construction_data = crawl('https://www.indeed.com/jobs?q=construction&l=Anywhere', 15)
writeJson('construction jobs.json', construction_data)

In [ ]:
security_data = crawl('https://www.indeed.com/jobs?q=security&l=Anywhere', 15)
writeJson('security jobs.json', security_data)

In [ ]:
tutoring_data = crawl('https://www.indeed.com/jobs?q=tutor&l=Anywhere', 15)
writeJson('tutoring jobs.json', tutoring_data)

In [ ]:
pharmacist_data = crawl('https://www.indeed.com/jobs?q=pharmacist&l=Anywhere', 15)
writeJson('pharmacist jobs.json', pharmacist_data)

In [ ]:
data_data = crawl('https://www.indeed.com/jobs?q=data+analyst&l=Anywhere', 15)
writeJson('data jobs.json', data_data)

In [ ]:
economist_data = crawl('https://www.indeed.com/jobs?q=economist&l=Anywhere', 15)
writeJson('economist jobs.json', economist_data)

In [ ]:
marketing_data = crawl('https://www.indeed.com/jobs?q=marketing&l=Anywhere', 15)
writeJson('marketing jobs.json', marketing_data)

In [ ]:
banker_data = crawl('https://www.indeed.com/jobs?q=banker&l=Anywhere', 15)
writeJson('banker jobs.json', banker_data)

In [ ]:
data= pd.read_json('accounting jobs.json')
data